In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

import os
from tqdm import tqdm_notebook

# 작업파일 불러오기

- 전체 :  ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시', '갤럭시케이스',
          '안경','모자','하이힐','후드','아이폰케이스','아이폰','자켓','청바지','점퍼','중고차','키보드','노트북','마우스','목도리',
          '네일','니트','목걸이','나이키','닌텐도','원피스','패딩','바지','향수','냉장고','레고','반지','셔츠','치마','스피커','정장',
          '선글라스','티셔츠','트레이닝','티비','지갑','세탁기','시계']
- '아디다스 0','에어컨 1','에어팟 2','오토바이 3','블라우스 4','팔찌 5','카메라 6','코트7','컴퓨터 8','구두 9','귀걸이10','갤럭시 11'

In [3]:
#내 작업
query_list = ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시']

In [4]:
total_df = []
for i in range(len(query_list)):
    total_df.append(pd.read_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/info/{query_list[i]}.csv'))

# 태그 추천 테스트
1. 문장-단어 TF-IDF 구하기 : 한 문장(product)에 중요도가 높은 단어들 테이블
2. 문장-문장 유사도 구하기 : 특정 문장(prodcut)기준 문장간 유사도를 통해 특정 문장과 유사한 문장 뽑기
3. 1을 미리 구해놓음 -> 2에서 비슷한 문장 뽑음 -> 2에서 뽑힌 문장을 1에서 찾음 -> 1에서 태그 뽑기


- 구현에 필요한 내용
 - 1,2번 구현 테스트
 - 언어 전처리 필요

In [5]:
adidas = total_df[0].copy()
adidas.fillna(0,inplace=True)
adidas.replace(0,'',inplace=True)
adidas['merge'] = adidas['title'] + adidas['keyword'] + adidas['cat1'] + adidas['cat2'] + adidas['cat3']
adidas

,product_id,title,keyword,cat1,cat2,cat3,view,image,merge
0,162427299,아디다스 문디알 풋살화265m,"아디다스,문디알,코파문디알,아디다스문디알,풋살화",스포츠/레저,축구,축구 용품,641,https://media.bunjang.co.kr/product/162427299_...,"아디다스 문디알 풋살화265m아디다스,문디알,코파문디알,아디다스문디알,풋살화스포츠/..."
1,172688578,아디다스 반팔티,"아디다스,아디다스반팔,반팔",남성의류,티셔츠,반팔 티셔츠,12,https://media.bunjang.co.kr/product/172688578_...,"아디다스 반팔티아디다스,아디다스반팔,반팔남성의류티셔츠반팔 티셔츠"
2,174560842,아디다스 봄가을 트레이닝 1230,100,남성의류,바지,기타(바지),4,https://media.bunjang.co.kr/product/174560842_...,아디다스 봄가을 트레이닝 1230100남성의류바지기타(바지)
3,171688500,아디다스 아이바 글러브 레드,"복싱,복싱글러브,아이바",스포츠/레저,기타 구기 스포츠,0,101,https://media.bunjang.co.kr/product/171688500_...,"아디다스 아이바 글러브 레드 복싱,복싱글러브,아이바스포츠/레저기타 구기 스포츠0"
4,174560837,아디다스 빅로고 버건디 후드티,"아디다스빅로고,아디다스후드티,빅로고후드,버건디후드",여성의류,후드티/후드집업,0,1,https://media.bunjang.co.kr/product/174560837_...,"아디다스 빅로고 버건디 후드티아디다스빅로고,아디다스후드티,빅로고후드,버건디후드여성의..."
...,...,...,...,...,...,...,...,...,...
5995,170889615,아디다스 x 고스티드+ 265 FG 최상급 축구화,"축구화,축구,아디다스x고스티드+,x고스티드,엑스",스포츠/레저,축구,축구 용품,390,https://media.bunjang.co.kr/product/170889615_...,"아디다스 x 고스티드+ 265 FG 최상급 축구화축구화,축구,아디다스x고스티드+,x..."
5996,118288417,아디다스 유로파 핑크 하의 85,"아디다스 유로파,유로파 85,유로파 핑크",여성의류,트레이닝,하의,291,https://media.bunjang.co.kr/product/118288417_...,"아디다스 유로파 핑크 하의 85아디다스 유로파,유로파 85,유로파 핑크여성의류트레이닝하의"
5997,118288210,아디다스 유로파 핑크 상의 85,"아디다스 유로파,85,유로파 핑크,아디다스 여자 상",여성의류,트레이닝,상의,152,https://media.bunjang.co.kr/product/118288210_...,"아디다스 유로파 핑크 상의 85아디다스 유로파,85,유로파 핑크,아디다스 여자 상여..."
5998,174460778,<240>아디다스 오즈위고 스웨이드 화이트 새상품,"아디다스,오즈위고,240,EG0546,화이트",신발,스니커즈,0,9,https://media.bunjang.co.kr/product/174460778_...,"<240>아디다스 오즈위고 스웨이드 화이트 새상품아디다스,오즈위고,240,EG054..."


In [6]:
test = adidas[['product_id','merge']]
test

,product_id,merge
0,162427299,"아디다스 문디알 풋살화265m아디다스,문디알,코파문디알,아디다스문디알,풋살화스포츠/..."
1,172688578,"아디다스 반팔티아디다스,아디다스반팔,반팔남성의류티셔츠반팔 티셔츠"
2,174560842,아디다스 봄가을 트레이닝 1230100남성의류바지기타(바지)
3,171688500,"아디다스 아이바 글러브 레드 복싱,복싱글러브,아이바스포츠/레저기타 구기 스포츠0"
4,174560837,"아디다스 빅로고 버건디 후드티아디다스빅로고,아디다스후드티,빅로고후드,버건디후드여성의..."
...,...,...
5995,170889615,"아디다스 x 고스티드+ 265 FG 최상급 축구화축구화,축구,아디다스x고스티드+,x..."
5996,118288417,"아디다스 유로파 핑크 하의 85아디다스 유로파,유로파 85,유로파 핑크여성의류트레이닝하의"
5997,118288210,"아디다스 유로파 핑크 상의 85아디다스 유로파,85,유로파 핑크,아디다스 여자 상여..."
5998,174460778,"<240>아디다스 오즈위고 스웨이드 화이트 새상품아디다스,오즈위고,240,EG054..."


## 불용어 처리

In [10]:
from konlpy.tag import Okt
import re

In [11]:
test2=test.copy()
test2= test2[:3]
test2

,product_id,merge
0,162427299,"아디다스 문디알 풋살화265m아디다스,문디알,코파문디알,아디다스문디알,풋살화스포츠/..."
1,172688578,"아디다스 반팔티아디다스,아디다스반팔,반팔남성의류티셔츠반팔 티셔츠"
2,174560842,아디다스 봄가을 트레이닝 1230100남성의류바지기타(바지)


In [8]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = [], test = False):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", review)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_review = okt.morphs(review_text, stem=True)
    
    if test:
        print(review_text)
        print(word_review)

    if remove_stopwords:
        
        # 불용어 제거(선택적)
        word_review = [token for token in word_review if not token in stop_words]

    return word_review

In [9]:
stop_words = [ '은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']

In [12]:
okt = Okt()
info=[]
for word in tqdm_notebook(test2['merge']):
    info.append(preprocessing(word, okt, remove_stopwords = True, stop_words=stop_words))

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_870/1835991182.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for word in tqdm_notebook(test2['merge']):


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
len(info)

3

In [14]:
info

[['아디다스',
  '문디알',
  '풋살',
  '화',
  '아디다스',
  '문디알',
  '코파',
  '문디알',
  '아디다스',
  '문디알',
  '풋살',
  '화',
  '스포츠',
  '레저',
  '축구',
  '축구',
  '용품'],
 ['아디다스',
  '반',
  '팔',
  '티',
  '아디다스',
  '아디다스',
  '반',
  '팔',
  '반',
  '팔',
  '남성',
  '의류',
  '티셔츠',
  '반',
  '팔',
  '티셔츠'],
 ['아디다스', '봄', '가을', '트레이닝', '남성', '의류', '바지', '기타', '바지']]

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
countvect = vect.fit_transform(info)
countvect.toarray()

AttributeError: 'list' object has no attribute 'lower'